<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Imp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
𝐐𝐮𝐞𝐬𝐭𝐢𝐨𝐧:
Given a DataFrame containing sales data, write a PySpark script to:
Replace missing values in the price column with the median price instead of the mean.
Drop rows where the product column is null, but if the price column is above 300, replace the null product with "Unknown".
Fill missing values in the quantity column with the average quantity rounded to the nearest integer.
Add a new column, total_value, which is the product of price and quantity.
Remove the product_id column from the DataFrame.

𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
data = [
 (1, "Laptop", 1000, 5),
 (2, "Mouse", None, None),
 (3, "Keyboard", 50, 2),
 (4, "Monitor", 200, None),
 (5, None, 500, None),
]
columns = ["product_id", "product", "price", "quantity"]


In [25]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName('learning').getOrCreate()

data=[(1, "Laptop", 1000, 5),(2, "Mouse", None, None),(3, "Keyboard", 50, 2),(4, "Monitor", 200, None),(5, None, 500, None)]

schema=["product_id", "product", "price", "quantity"]

df=spark.createDataFrame(data,schema)

df.show()

median_price=df.select(median(col('price'))).collect()[0][0]

avg_quantity=df.select(round(avg(col('quantity')))).collect()[0][0]

print(avg_quantity)

df1=df.fillna({'price':median_price,'quantity':avg_quantity})

df1.show()

#df2=df1.withColumn('product',when(col('price') > 300 & col('product').isNotNull()  ,'Unknown').otherwise(col('product')))

df2 = df1.withColumn('product',
                    when((col('price') > 300) & (col('product').isNull()), 'Unknown')
                    .otherwise(col('product'))).filter(col('product').isNotNull()).drop(col('product_id'))

df2.show()

df3=df2.withColumn('total_value',col('price')*col('quantity'))

df3.show()



+----------+--------+-----+--------+
|product_id| product|price|quantity|
+----------+--------+-----+--------+
|         1|  Laptop| 1000|       5|
|         2|   Mouse| NULL|    NULL|
|         3|Keyboard|   50|       2|
|         4| Monitor|  200|    NULL|
|         5|    NULL|  500|    NULL|
+----------+--------+-----+--------+

4.0
+----------+--------+-----+--------+
|product_id| product|price|quantity|
+----------+--------+-----+--------+
|         1|  Laptop| 1000|       5|
|         2|   Mouse|  350|       4|
|         3|Keyboard|   50|       2|
|         4| Monitor|  200|       4|
|         5|    NULL|  500|       4|
+----------+--------+-----+--------+

+--------+-----+--------+
| product|price|quantity|
+--------+-----+--------+
|  Laptop| 1000|       5|
|   Mouse|  350|       4|
|Keyboard|   50|       2|
| Monitor|  200|       4|
| Unknown|  500|       4|
+--------+-----+--------+

+--------+-----+--------+-----------+
| product|price|quantity|total_value|
+--------+-----+-